In [1]:
import numpy as np

from tqdm import tqdm as progressbar

from matplotlib import pyplot as plt
from matplotlib import rc

rc('font', size=12)
rc('axes', titlesize=14)
rc('axes', labelsize=14)

%matplotlib inline
%config InlineBackend.figure_format='retina'

# CORE FUNCTIONS

In [2]:
def compute_snr(distribution, med, mad):
    return (np.max(distribution) - med) / (mad * 1.48)

def compute_mad(array, med):
    series_mad = np.abs(array - med)
    return np.median(series_mad)

def compute_mom(L):
    if len(L) < 10:
        L.sort()
        return L[int(len(L) / 2)]
    S = []
    l_index = 0

    for l_index in range(0, len(L) - 1, 5):
        S.append(L[l_index:l_index + 5])

    S.append(L[l_index:])
    meds = []

    for sub_list in S:
        meds.append(compute_mom(sub_list))

    L2 = compute_mom(meds)
    L1 = L3 = []

    for i in L:
        if i < L2:
            L1.append(i)
        if i > L2:
            L3.append(i)

    if len(L) < len(L1):
        return compute_mom(L1)

    elif len(L) > len(L1) + 1:
        return compute_mom(L3)

    else:
        return L2

def downsample(array, downsampling):
    if downsampling == 0:
        return array

    series = []
    for i in range(0, len(array), downsampling):
        series.append(np.mean(array[i:i+downsampling]))
    return np.asarray(series)


def draw(n_samples=2500):
    return np.random.normal(0, 1, n_samples)

# MAIN

In [3]:
# Simple calls
print ("i", "j", "mom", "mad", "snr")
for j in range(10, 2500, 15):
    for i in range(1000):
        samples = draw(j)
        mom = compute_mom(samples)
        mad = compute_mad(samples, mom)
        snr = compute_snr(samples, mom, mad)
        
        if mad == 0:
            print (samples, i, j, mom, mad, snr)

i j mom mad snr


In [ ]:
from multiprocessing import Pool

def SNR(n_samples):
    for i in range(1000):
        samples = draw(n_samples)
        median = np.median(samples)
        std = np.std(samples)
        mom = compute_mom(samples)
        mad = compute_mad(samples, mom)

        snr_median_mad = compute_snr(samples, median, mad)
        snr_mom_mad = compute_snr(samples, mom, mad)
        snr_median_std = (np.max(samples)-median)/std
        snr_mom_std = (np.max(samples)-mom)/std

with Pool(processes=10) as pool:
    pool.map(SNR, range(10, 2500, 15))